In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from pathlib import Path
import h5py

ModuleNotFoundError: No module named 'h5py'

In [3]:
atlas_dir= Path("../DiarrheaExample/")

gene_abundance_file= atlas_dir/"Genecatalog/counts/median_coverage.h5"

with h5py.File(gene_abundance_file, 'r') as hdf_file:

    # data_matrix = hdf_file['data'][:]
    sample_names = hdf_file['data'].attrs['sample_names'].astype(str)

NameError: name 'h5py' is not defined